In [20]:
from gurobipy import *
import pandas as pd
import numpy as np

## Parameters

In [21]:
df_county = pd.read_csv('data/county_code.csv')
df_market = pd.read_csv('data/market_code.csv')
I = range(df_county.shape[0])
J = range(df_market.shape[0])

In [22]:
df_demand = pd.read_excel("data/demand_function.xlsx")
A = df_demand["slope"]
B = df_demand["intercept"]
# df_demand

In [47]:
# 旺季產量
df_production = pd.read_csv("data/county_production.csv")
# 旺季一個月
df_production1 = df_production.copy()
df_production1['quantity'] = df_production1['quantity'] * 0.6 / 4 
# 每天
df_production1['quantity'] /= 30
Q_s = df_production1['quantity']
df_production1

,Unnamed: 0,name,quantity,code
0,0,新北市,15366.243889,0
1,1,台北市,2748.499444,1
2,2,桃園市,16033.365000,2
3,3,台中市,176370.740556,3
4,4,台南市,95918.435556,4
5,5,高雄市,38387.124444,5
6,6,宜蘭縣,431689.425000,6
7,7,新竹縣,37136.298333,7
8,8,苗栗縣,23024.741667,8
9,9,彰化縣,226593.761111,9


In [53]:
L = 2500
C = 1.53
M = 6800

In [58]:
Q_s

0      15366.243889
1       2748.499444
2      16033.365000
3     176370.740556
4      95918.435556
5      38387.124444
6     431689.425000
7      37136.298333
8      23024.741667
9     226593.761111
10    233545.821111
11    458767.212222
12    117518.770556
13     74353.701667
14     31505.105000
15     19572.552778
16        23.720000
17       603.262222
18      1003.147222
19     12541.462500
Name: quantity, dtype: float64

- Formulation1
$$
\begin{split}
 \mbox{max} \quad & \sum_{i \in I}\sum_{j \in J}(A_jx_{ij} + B_j)x_{ij} - C(My_{ij} + x_{ij}) \quad \mbox{(revenue)} \\
 \mbox{s.t.} \quad & \sum_j x_{ij} \leq Q^s_i \quad \forall i \in I \quad \mbox{(upper limit of production quantity)}\\
 & x_{ij} \leq y_{ij}L \quad \forall i \in I, j \in J \quad \mbox{(compute number of truck)}\\
 & x_{ij} \geq 0 \quad \forall i \in I, j \in J \quad \mbox{(Sign Constraint)} \\
 & y_{ij} \in \mathbb{Z}^+ \quad \forall i \in I, j \in J \quad \mbox{(Sign Constraint)} \\
\end{split}
$$

In [54]:
model = Model('solution1')

# decision variables
x = model.addVars(I, J, lb = 0, vtype = GRB.CONTINUOUS, name = 'x')
y = model.addVars(I, J, lb = 0, vtype = GRB.INTEGER, name = 'y')

# objective function
model.setObjective(quicksum((A[j] * x[i, j] + B[j]) * x[i, j]
                            - C * (M * y[i, j] + x[i, j]) for i in I for j in J), GRB.MAXIMIZE)

# constraints (for both single and multiple sourcing)
constr = model.addConstrs(quicksum(x[i, j] for j in J) <= Q_s[i] for i in I)
constr = model.addConstrs(x[i, j] <= y[i, j] * L for i in I for j in J)

model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 360 rows, 680 columns and 1020 nonzeros
Model fingerprint: 0x3cab83f5
Model has 340 quadratic objective terms
Variable types: 340 continuous, 340 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [3e+03, 5e+05]
  QObjective range [1e+02, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 5e+05]
Found heuristic solution: objective -0.0000000
Presolve removed 342 rows and 646 columns
Presolve time: 0.07s
Presolved: 18 rows, 34 columns, 51 nonzeros
Presolved model has 17 quadratic objective terms
Found heuristic solution: objective 5.857355e+08
Variable types: 17 continuous, 17 integer (0 binary)

Root relaxation: objective 6.179483e+08, 35 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds    

In [55]:
for var in model.getVars():
    if var.varName.startswith('x'):
        print(var.varName, '=', var.x)
    elif var.varName.startswith('y'):
        print(var.varName, '=', var.x)

x[0,0] = 35.500896583433544
x[0,1] = 36.37710557436537
x[0,2] = 21.50254190226171
x[0,3] = 21.76750039923347
x[0,4] = 22.192052060457303
x[0,5] = 31.3010265049856
x[0,6] = 12.393812503531272
x[0,7] = 28.490112184736326
x[0,8] = 37.84734994244368
x[0,9] = 26.412435572454584
x[0,10] = 19.780856769024503
x[0,11] = 29.15929701279042
x[0,12] = 36.1581297521033
x[0,13] = 68.01119123431354
x[0,14] = 80.31870719178082
x[0,15] = 49.08857339093029
x[0,16] = 30.237991401958443
x[1,0] = 35.500896583433544
x[1,1] = 36.37710557436537
x[1,2] = 21.50254190226171
x[1,3] = 21.76750039923347
x[1,4] = 22.192052060457303
x[1,5] = 31.3010265049856
x[1,6] = 12.393812503531272
x[1,7] = 28.490112184736326
x[1,8] = 37.84734994244368
x[1,9] = 26.412435572454584
x[1,10] = 19.780856769024503
x[1,11] = 29.15929701279042
x[1,12] = 36.1581297521033
x[1,13] = 68.01119123431354
x[1,14] = 80.31870719178082
x[1,15] = 49.08857339093029
x[1,16] = 30.237991401958443
x[2,0] = 35.500896583433544
x[2,1] = 36.37710557436537
x[2

In [56]:
df_result_production = df_county.copy()
p = []
for i in I:
    total = 0
    for j in J:
        total += x[i, j].x
    p.append(total)

In [57]:
p

[586.5395799808042,
 586.5395799808042,
 586.5395799808042,
 586.5395799808042,
 586.5395799808042,
 586.5395799808042,
 586.5395799808042,
 586.5395799808042,
 586.5395799808042,
 586.5395799808042,
 586.5395799808042,
 586.5395799808042,
 586.5395799808042,
 586.5395799808042,
 586.5395799808042,
 586.5395799808042,
 23.720000000000006,
 586.5395799808042,
 586.5395799808042,
 586.4519195350244]